In [5]:
import json
import sqlite3
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from categories import CATEGORY_MAPPINGS

categories = CATEGORY_MAPPINGS

conn = sqlite3.connect("../data/podcasts.db")
cursor = conn.cursor()

subcategory_counts = {}


def collect_subcategories(section):
    """Recursively collect all subcategories in a section."""
    if "subcategories" in section:
        for subcat_key, subcat_value in section["subcategories"].items():
            yield subcat_key, subcat_value


for top_level_key, top_level_value in categories.items():
    if "subcategories" in top_level_value:
        for subcat_key, subcat_value in collect_subcategories(top_level_value):
            match_categories = subcat_value.get("match_categories", [])
            total = 0

            for cat, subcat in match_categories:
                cursor.execute(
                    """
                    SELECT COUNT(*)
                    FROM episode
                    JOIN podcast ON episode.podcast_id = podcast.id
                    WHERE podcast.category = ? AND podcast.subcategory = ?
                    """,
                    (cat, subcat),
                )
                count = cursor.fetchone()[0]
                total += count

            subcategory_counts[subcat_key] = total
sorted_data = dict(sorted(subcategory_counts.items(), key=lambda item: item[1]))
with open("../working/frontend_subcategory_counts.json", "w") as f:
    json.dump(sorted_data, f)

conn.close()